<a href="https://colab.research.google.com/github/SamuelaAnastasi/PrivateAiChallenge_ToyFederatedLearning/blob/master/PrivateAiChallenge_ToyFederatedLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Toy Federated Learning
Start by training a toy model the centralized way. We need:

* a toy dataset
* a model
* some basic training logic for training a model to fit the data.


In [1]:
!pip install tf-encrypted

! URL="https://github.com/openmined/PySyft.git" && FOLDER="PySyft" && if [ ! -d $FOLDER ]; then git clone -b dev --single-branch $URL; else (cd $FOLDER && git pull $URL && cd ..); fi;

!cd PySyft; python setup.py install  > /dev/null

import os
import sys
module_path = os.path.abspath(os.path.join('./PySyft'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
!pip install --upgrade --force-reinstall lz4
!pip install --upgrade --force-reinstall websocket
!pip install --upgrade --force-reinstall websockets
!pip install --upgrade --force-reinstall zstd

     |████████████████████████████████| 1.4MB 2.8MB/s 
     |████████████████████████████████| 276kB 41.8MB/s 
  Stored in directory: /root/.cache/pip/wheels/16/27/a1/775c62ddea7bfa62324fd1f65847ed31c55dadb6051481ba3f
Successfully built pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Cloning into 'PySyft'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 28053 (delta 6), reused 13 (delta 4), pack-reused 28034
Receiving objects: 100% (28053/28053), 31.84 MiB | 23.15 MiB/s, done.
Resolving deltas: 100% (18529/18529), done.
zip_safe flag not set; analyzing archive contents...
zip_safe flag not set; analyzing archive contents...
__pycache__.zstd.cpython-36: module references __file__
     |████████████████████████████████| 389kB 2.8MB/s 
ERROR: syft 0.1.21a1 has requirement msgpack>=0.6.1, but you'll have msgpack 0.5

In [0]:
import torch as th
import syft as sy
from torch import nn, optim

In [0]:
hook = sy.TorchHook(th)

In [0]:
# Create local Dataset and target
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
target = th.tensor([[1.],[1], [0], [0]], requires_grad=True)

In [0]:
# Create a local linear Model
model = nn.Linear(2,1)

In [0]:
# define optimizer
opt = optim.SGD(params=model.parameters(), lr=0.1)

In [11]:
# define training function and train model locally
def train(iterations=20):
    for iter in range(iterations):
        opt.zero_grad()

        pred = model(data)

        loss = ((pred - target)**2).sum()

        loss.backward()

        opt.step()

        print(loss.data)
        
train()

tensor(1.4181)
tensor(0.2855)
tensor(0.1527)
tensor(0.1084)
tensor(0.0800)
tensor(0.0596)
tensor(0.0445)
tensor(0.0334)
tensor(0.0251)
tensor(0.0189)
tensor(0.0143)
tensor(0.0108)
tensor(0.0082)
tensor(0.0062)
tensor(0.0047)
tensor(0.0036)
tensor(0.0027)
tensor(0.0021)
tensor(0.0016)
tensor(0.0012)


In [0]:
# create 2 virtual workers
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")

In [0]:
#send data and target to bob
data_bob = data[0:2].send(bob)
target_bob = target[0:2].send(bob)

In [0]:
#send data and target to alice
data_alice = data[2:4].send(alice)
target_alice = target[2:4].send(alice)

In [0]:
datasets = [(data_bob, target_bob), (data_alice, target_alice)]

In [0]:
# define training function: model, optimizer 
def train(iterations=20):

    model = nn.Linear(2,1)
    opt = optim.SGD(params=model.parameters(), lr=0.1)
    
    for iter in range(iterations):

        for _data, _target in datasets:

            # send model to the data 
            model = model.send(_data.location)

            # train remote model
            opt.zero_grad()
            pred = model(_data)
            loss = ((pred - _target)**2).sum()
            loss.backward()
            opt.step()

            # get model back after training
            model = model.get()

            print(loss.get())

In [19]:
# train model
train()

tensor(4.2596, requires_grad=True)
tensor(1.6694, requires_grad=True)
tensor(0.8816, requires_grad=True)
tensor(1.0601, requires_grad=True)
tensor(0.4651, requires_grad=True)
tensor(0.6211, requires_grad=True)
tensor(0.2671, requires_grad=True)
tensor(0.3622, requires_grad=True)
tensor(0.1543, requires_grad=True)
tensor(0.2113, requires_grad=True)
tensor(0.0892, requires_grad=True)
tensor(0.1235, requires_grad=True)
tensor(0.0516, requires_grad=True)
tensor(0.0722, requires_grad=True)
tensor(0.0298, requires_grad=True)
tensor(0.0423, requires_grad=True)
tensor(0.0173, requires_grad=True)
tensor(0.0249, requires_grad=True)
tensor(0.0100, requires_grad=True)
tensor(0.0146, requires_grad=True)
tensor(0.0058, requires_grad=True)
tensor(0.0086, requires_grad=True)
tensor(0.0034, requires_grad=True)
tensor(0.0051, requires_grad=True)
tensor(0.0020, requires_grad=True)
tensor(0.0030, requires_grad=True)
tensor(0.0011, requires_grad=True)
tensor(0.0018, requires_grad=True)
tensor(0.0007, requi